In [74]:
%matplotlib inline
import loader
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy as sp
file = '2012-Consolidated-stripped.csv'

In [2]:
def reservoir_sample(iterator, k):
    """
    Basic reservoir sample. Takes a target sample amount
    """
    # fill the reservoir to start
    iterator = iter(iterator)
    result = [next(iterator) for _ in range(k)]
    n = k
    for item in iterator:
        n += 1
        s = random.randint(0, n)
        if s < k:
            result[s] = item
    return result


def get_sample_size(len1, len2, percent):
    return int(min(len1, len2) * percent)

In [3]:
reload(loader)
rows = loader.load_raw(file)

In [4]:
# split dataset by chronic / not_chronic
chron = []
notchron = []

for pid in rows:
    person = rows[pid]
    person_data = person.info
    chron_bool = False
    for code in person.info:
        if 'chronic_'in code and not chron_bool:
                if person_data[code] is 1:
                    chron_bool = True
                elif person_data[code] is -9 or person_data[code] is -8 or person_data[code] is -7:
                    rows[pid].chronic = -1
            
    if chron_bool:
        chron.append(pid)
    else:
        notchron.append(pid)

In [139]:
# sample helpers
def sample_two(l1, l2, pct):
    ssize = get_sample_size(len(l1), len(l2), pct)
    s1 = reservoir_sample(l1, ssize)
    s2 = reservoir_sample(l2, ssize)
    return s1, s2

def sample_one(single, pct):
    ssize = get_sample_size(len(single), sys.maxint, pct)
    sample = reservoir_sample(single, ssize)
    return sample

# spending getters
def get_spending(id_list, rows):
    return [rows[pid].info['spending_dist_total'] for pid in id_list]

def get_subsidized(id_list, rows):
    return [rows[pid].info['spending_pay_medicaid'] + rows[pid].info['spending_pay_medicare'] for pid in id_list]

def get_medicaid(id_list, rows):
    return [rows[pid].info['spending_pay_medicaid'] for pid in id_list]

def get_medicare(id_list, rows):
    return [rows[pid].info['spending_pay_medicare'] for pid in id_list]

# calculate confidence interval
def get_ci(samples, level):
    mean = np.mean(samples)
    scale = np.std(samples)
    ci = stats.norm.interval(level, loc = mean, scale = scale)
    return ci

In [201]:
# create samples of chronic / not chronic
schron, snotchron = sample_two(chron, notchron, 0.7)

In [202]:
# get average total spending for both groups
schron_spend = get_spending(schron, rows)
snotchron_spend = get_spending(snotchron, rows)
print 'chronic total spending', np.average(schron_spend), np.std(schron_spend)
print 'not chronic total spending', np.average(snotchron_spend), np.std(snotchron_spend)

chronic total spending 6271.8693077 15814.0481046
not chronic total spending 1466.77997864 10609.2016612


In [203]:
# get average subsidized spending
schron_sspend = get_subsidized(schron, rows)
snotchron_sspend = get_subsidized(snotchron, rows)
print 'chronic sub spending', np.average(schron_sspend), np.std(schron_sspend)
print 'not chronic sub spending', np.average(snotchron_sspend), np.std(snotchron_sspend)

chronic sub spending 2895.02903194 10409.1677886
not chronic sub spending 379.372172056 2960.2254977


In [204]:
# get total visits
schron_office = [rows[pid].info['service_office'] for pid in schron]
snotchron_office = [rows[pid].info['service_office'] for pid in snotchron]

# get cost per visit
schron_cpv = [rows[pid].info['spending_dist_office']/rows[pid].info['service_office'] for pid in schron if rows[pid].info['service_office']>0]
snotchron_cpv = [rows[pid].info['spending_dist_office']/rows[pid].info['service_office'] for pid in snotchron if rows[pid].info['service_office']>0]

In [205]:
print 'chronic office visits', np.average(schron_office), np.std(schron_office)
print 'not chronic office visits', np.average(snotchron_office), np.std(snotchron_office)

print 'chronic cost per visit', np.average(schron_cpv), np.std(schron_cpv)
print 'not chronic cost per visit', np.average(snotchron_cpv), np.std(snotchron_cpv)

chronic office visits 6.41042819691 11.7457204847
not chronic office visits 2.00835032527 5.11283766878
chronic cost per visit 3.9145456781 33.103308646
not chronic cost per visit 1.38003309432 16.3699163963


In [11]:
# split dataset between chronic conditions
hbp = []
coronary = []
myocardial = []
stroke = []
diabetes = []
asthma = []
arthritis = []
cancer = []

hbp2 = []
multiple = []

diseases = {
    'chronic_hbp': hbp, 
    'chronic_coronary': coronary, 
    'chronic_myocardial': myocardial, 
    'chronic_stroke': stroke,
    'chronic_diabetes': diabetes,
    'chronic_asthma': asthma,
    'chronic_arthritis': arthritis, 
    'chronic_cancer': cancer
}

for pid in rows:
    person = rows[pid]
    person_data = person.info
    has_chronic = False
    for code in person.info:
        if 'chronic_hbp2' in code and person_data[code] is 1:
            hbp2.append(pid)
            continue
        if code in diseases and person_data[code] is 1: 
            diseases[code].append(pid)
            if not has_chronic:
                has_chronic = True
            else:
                multiple.append(pid)
                

# This boolean controls whether the multiple condition group is mutually exclusive from individual
separate = False
if (separate):
    # get people who only have that condition
    set_multiple = set(multiple)
    for disease in diseases:
        setify = set(diseases[disease])
        diseases[disease] = list(setify.difference(set_multiple))

    # get people who only have hbp and who were diagnosed twice
    set_hbp = set(hbp)
    set_hbp2 = set(hbp2)
    hbp2 = list(set_hbp.intersection(set_hbp2))   

In [149]:
# create sub samples of each condition
shbp = sample_one(hbp, 0.7)
scoronary = sample_one(coronary, 0.7)
smyocardial = sample_one(myocardial, 0.7)
sstroke = sample_one(stroke, 0.7)
sdiabetes = sample_one(diabetes, 0.7)
sasthma = sample_one(asthma, 0.7)
sarthritis = sample_one(arthritis, 0.7)
scancer = sample_one(cancer, 0.7)

shbp2 = sample_one(hbp2, 0.7)
smultiple = sample_one(multiple, 0.7)

In [150]:
# get subsidized spending for each condition (filtering out multiple)
shbp_ssp = get_subsidized(shbp, rows)
scoronary_ssp = get_subsidized(scoronary, rows)
smyocardial_ssp = get_subsidized(smyocardial, rows)
sstroke_ssp = get_subsidized(sstroke, rows)
sdiabetes_ssp = get_subsidized(sdiabetes, rows)
sasthma_ssp = get_subsidized(sasthma, rows)
sarthritis_ssp = get_subsidized(sarthritis, rows)
scancer_ssp = get_subsidized(scancer, rows)

shbp2_ssp = get_subsidized(shbp2, rows)
smultiple_ssp = get_subsidized(smultiple, rows)

print 'hbp', np.average(shbp_ssp)
print 'coronary', np.average(scoronary_ssp)
print 'myocardial', np.average(smyocardial_ssp)
print 'stroke', np.average(sstroke_ssp)
print 'diabetes', np.average(sdiabetes_ssp)
print 'asthma', np.average(sasthma_ssp)
print 'arthritis', np.average(sarthritis_ssp)
print 'cancer', np.average(scancer_ssp)

print 'hbp2', np.average(shbp2_ssp)
print 'multiple', np.average(smultiple_ssp)

hbp 3758.78559949
coronary 8180.85251046
myocardial 9533.86068111
stroke 9063.47313433
diabetes 5568.9761039
asthma 2502.32476994
arthritis 4484.58450218
cancer 5309.37359199
hbp2 4291.1878464
multiple 6740.47309598


In [151]:
all_s_sp = shbp_ssp + scoronary_ssp + smyocardial_ssp + sstroke_ssp + sdiabetes_ssp + sasthma_ssp + sarthritis_ssp + scancer_ssp
avg_all_s_sp = np.average(all_s_sp)
print 'diabetes patients spent', 100*(np.average(sdiabetes_ssp) - avg_all_s_sp)/avg_all_s_sp, '% more than average'

diabetes patients spent 19.1625310374 % more than average


In [152]:
cared, uncared = [], []
for pid in diabetes:
    person = rows[pid]
    care = 0
    if ('diabetes_a1c' in person.info and person.info['diabetes_a1c']>0):
        care += 1
    if ('diabetes_foot' in person.info and person.info['diabetes_foot'] > 0):
        care += 1
    if ('diabetes_eye' in person.info and person.info['diabetes_eye'] > 0):
        care += 1
    if ('diabetes_cholest' in person.info and person.info['diabetes_cholest'] > 0):
        care += 1
    if (care <= 1):
        uncared.append(pid)
    else:
        cared.append(pid)

In [153]:
print len(cared), len(uncared)

2261 490


In [188]:
# get samples for cared and uncared group
scared, suncared = sample_two(cared, uncared, 0.7)
cared_cost = get_subsidized(scared,rows)
uncared_cost = get_subsidized(suncared, rows)
cared_msp = get_medicaid(scared,rows)
uncared_msp = get_medicaid(suncared, rows)
cared_mcsp = get_medicare(scared,rows)
uncared_mcsp = get_medicare(suncared, rows)

In [189]:
print np.average(uncared_cost), np.average(cared_cost)
print np.average(uncared_msp), np.average(cared_msp)
print np.average(uncared_mcsp), np.average(cared_mcsp)

7144.90670554 4279.42565598
1925.7696793 927.516034985
5219.13702624 3351.90962099


In [190]:
# get confidence intervals:
ci_uncared = get_ci(uncared_cost, 0.05)
ci_cared = get_ci(cared_cost, 0.05)
ci_uc_msp = get_ci(uncared_msp, 0.05)
ci_c_msp = get_ci(cared_msp, 0.05)
ci_uc_mcsp = get_ci(uncared_mcsp, 0.05)
ci_c_mcsp = get_ci(cared_mcsp, 0.05)
print "spending", "uncared", "cared"
print "subsidized", ci_uncared, ci_cared
print "medicaid", ci_uc_msp, ci_c_msp
print "medicare", ci_uc_mcsp, ci_c_mcsp

spending uncared cared
subsidized (5665.0834126515065, 8624.7299984272122) (3740.4547936093595, 4818.3965183439932)
medicaid (1386.1580265845175, 2465.3813320160652) (717.83658065179179, 1137.1954893190536)
medicare (3895.4885281755478, 6542.785524302587) (2852.1559571725952, 3851.6632848099121)


In [198]:
avg_all = (np.average(uncared_cost)+np.average(cared_cost))/2
avg_msp = (np.average(uncared_msp)+np.average(cared_msp))/2
avg_mcsp = (np.average(uncared_mcsp)+np.average(cared_mcsp))/2
print "The uncared group spend", 100*(np.average(uncared_cost)-avg_all)/avg_all, "% more than average"
print "The uncared group spend", 100*(np.average(uncared_msp)-avg_msp)/avg_msp, "% more medicaid than average."
print "The uncared group spend", 100*(np.average(uncared_mcsp)-avg_mcsp)/avg_mcsp, "% more medicare than average."

The uncared group spend 25.0822626556 % more than average
The uncared group spend 34.9861087979 % more medicaid than average.
The uncared group spend 21.785290433 % more medicare than average.


In [193]:
# age breakdown for cared and uncared groups
cared_young, cared_mid, cared_old = [], [], []
uncared_young, uncared_mid, uncared_old = [], [], []
for pid in scared:
    person = rows[pid]
    if person.info['demo_age']>=65:
        cared_old.append(pid)
    elif person.info['demo_age']>=45:
        cared_mid.append(pid)
    elif person.info['demo_age']>18:
        cared_young.append(pid)

for pid in suncared:
    person = rows[pid]
    if person.info['demo_age']>=65:
        uncared_old.append(pid)
    elif person.info['demo_age']>=45:
        uncared_mid.append(pid)
    elif person.info['demo_age']>18:
        uncared_young.append(pid)

In [194]:
cared_old_cost = get_subsidized(cared_old, rows)
cared_mid_cost = get_subsidized(cared_mid, rows)
cared_young_cost = get_subsidized(cared_young, rows)
uncared_old_cost = get_subsidized(uncared_old, rows)
uncared_mid_cost = get_subsidized(uncared_mid, rows)
uncared_young_cost = get_subsidized(uncared_young, rows)

In [195]:
print 'Subsidized', 'Old', 'Mid', 'Young'
print 'cared', np.average(cared_old_cost), np.average(cared_mid_cost), np.average(cared_young_cost)
print 'uncared', np.average(uncared_old_cost), np.average(uncared_mid_cost), np.average(uncared_young_cost)

Subsidized Old Mid Young
cared 6503.12765957 2531.7278481 3429.29545455
uncared 9451.53076923 4516.6884058 1310.62790698


In [199]:
# get confidence intervals:
ci_uncared_o = get_ci(uncared_old_cost, 0.05)
ci_cared_o = get_ci(cared_old_cost, 0.05)
ci_uncared_m = get_ci(uncared_mid_cost, 0.05)
ci_cared_m = get_ci(cared_mid_cost, 0.05)
ci_uncared_y = get_ci(uncared_young_cost, 0.05)
ci_cared_y = get_ci(cared_young_cost, 0.05)
print "", "uncared", "cared"
print "old", ci_uncared_o, ci_cared_o
print "mid", ci_uncared_m, ci_cared_m
print "young", ci_uncared_y, ci_cared_y

 uncared cared
old (7584.0618708189741, 11318.999667642564) (5848.8260966943226, 7157.4292224546143)
mid (3540.6605324619186, 5492.7162791322835) (2126.6903438941017, 2936.76535230843)
young (1050.664103363089, 1570.5917105903995) (2987.4679112778194, 3871.1229978130896)


In [200]:
m = (np.average(uncared_old_cost) + np.average(cared_old_cost))/2
print "The un100*(np.average(uncared_old_cost)-m)/m

0.184798886345
